Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [50]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.721941
Minibatch accuracy: 6.2%
Validation accuracy: 11.6%
Minibatch loss at step 50: 1.814844
Minibatch accuracy: 50.0%
Validation accuracy: 41.6%
Minibatch loss at step 100: 0.978562
Minibatch accuracy: 62.5%
Validation accuracy: 62.0%
Minibatch loss at step 150: 1.269531
Minibatch accuracy: 62.5%
Validation accuracy: 74.7%
Minibatch loss at step 200: 0.974083
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.858127
Minibatch accuracy: 75.0%
Validation accuracy: 76.5%
Minibatch loss at step 300: 0.407917
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 350: 0.706703
Minibatch accuracy: 68.8%
Validation accuracy: 80.0%
Minibatch loss at step 400: 1.202396
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.312999
Minibatch accuracy: 93.8%
Validation accuracy: 82.4%
Minibatch loss at step 500: 0.453747
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_nodes= 1024
eval_batch_size = 100

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(eval_batch_size, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(eval_batch_size, image_size, image_size, num_channels))
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

def eval_in_batches(dataset, placeholder, predictor):
    pred = []
    for step in range(100):
        offset = (step * eval_batch_size) % (valid_labels.shape[0] - eval_batch_size)
        batch_data = dataset[offset:(offset + eval_batch_size), :, :, :]
        feed_dict = {placeholder : batch_data}
        batch = session.run(
          predictor, feed_dict=feed_dict)
        for x in batch:
            pred.append(x.tolist())
    return np.array(pred)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0 and step != 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        eval_in_batches(valid_dataset, tf_valid_dataset, valid_prediction), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(
      eval_in_batches(test_dataset, tf_test_dataset, test_prediction), test_labels))


Initialized
Minibatch loss at step 50: 2.087838
Minibatch accuracy: 43.8%
Validation accuracy: 39.2%
Minibatch loss at step 100: 1.192040
Minibatch accuracy: 31.2%
Validation accuracy: 62.4%
Minibatch loss at step 150: 1.302548
Minibatch accuracy: 62.5%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.874619
Minibatch accuracy: 81.2%
Validation accuracy: 75.5%
Minibatch loss at step 250: 0.610278
Minibatch accuracy: 87.5%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.510534
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.868640
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 400: 1.154776
Minibatch accuracy: 68.8%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.356934
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.419860
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Minibatch loss at step 550: 0.721722
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [15]:
patch_size = 5
depth = 20
num_hidden = 100
num_channels = 1
num_labels = 10
batch_size = 10
image_size = 28

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_valid_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth*2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth*2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [4 * 4 * depth*2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, dropout=None):
    conv = tf.nn.relu(tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID') + layer1_biases)
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    conv = tf.nn.relu(tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='VALID') + layer2_biases)
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    if dropout:
        dropout_layer = tf.nn.dropout(hidden, dropout)
        return tf.matmul(dropout_layer, layer4_weights) + layer4_biases
    else:
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [19]:
num_epochs = 40

def eval_in_batches(dataset, placeholder, predictor):
    pred = []
    total_batch = int(len(dataset) // batch_size) 
    for step in range(total_batch):
        offset = (step * batch_size) 
        batch_data = dataset[offset:(offset + batch_size), :, :, :]
        feed_dict = {placeholder : batch_data}
        batch = session.run(
          predictor, feed_dict=feed_dict)
        for x in batch:
            pred.append(x.tolist())
    return np.array(pred)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  total_batch = int(len(train_dataset) // batch_size)
  for epoch in range(num_epochs):
    cost = 0
    for i in range(total_batch):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (i * batch_size) % (train_labels.shape[0])
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :,  :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        cost += l 
        if i%(10000-1) == 0 :
            print("Minibatch accuracy at step %d: %.2f%%" % (i+1, accuracy(predictions, batch_labels)))
    print("Average loss at epoch %d: %f" % (epoch+1, cost/total_batch))
    print("Validation accuracy: %.2f%%" % accuracy(eval_in_batches(valid_dataset, tf_valid_dataset, valid_prediction), valid_labels))
  print("Test accuracy: %.2f%%" % accuracy(eval_in_batches(test_dataset, tf_test_dataset, test_prediction), test_labels))

Initialized
Minibatch accuracy at step 1: 0.00%
Minibatch accuracy at step 10000: 90.00%
Minibatch accuracy at step 19999: 70.00%
Average loss at epoch 1: 0.555972
Validation accuracy: 88.73%
Minibatch accuracy at step 1: 90.00%
Minibatch accuracy at step 10000: 90.00%
Minibatch accuracy at step 19999: 80.00%
Average loss at epoch 2: 0.415898
Validation accuracy: 89.16%
Minibatch accuracy at step 1: 90.00%
Minibatch accuracy at step 10000: 80.00%
Minibatch accuracy at step 19999: 80.00%
Average loss at epoch 3: 0.383735
Validation accuracy: 90.00%
Minibatch accuracy at step 1: 90.00%
Minibatch accuracy at step 10000: 90.00%
Minibatch accuracy at step 19999: 90.00%
Average loss at epoch 4: 0.363324
Validation accuracy: 90.17%
Minibatch accuracy at step 1: 90.00%
Minibatch accuracy at step 10000: 90.00%
Minibatch accuracy at step 19999: 80.00%
Average loss at epoch 5: 0.347807
Validation accuracy: 90.31%
Minibatch accuracy at step 1: 90.00%
Minibatch accuracy at step 10000: 80.00%
Miniba